# Tourism #
We are interested in producing hierarchical reconciled forecasts for the Tourism dataset.

This example is based on chapter 11 of:

Hyndman, R.J., & Athanasopoulos, G. (2021) *Forecasting: principles and practice, 
3rd edition*, OTexts: Melbourne, Australia. OTexts.com/fpp3. Accessed on 05/07/2022.

The data was taken from R's tsibble package:

Wang, E, D Cook, and RJ Hyndman (2020). *A new tidy data structure to support exploration 
and modeling of temporal data*, Journal of Computational and Graphical Statistics, 29:3, 
466-478, doi:10.1080/10618600.2019.1695624.

https://tsibble.tidyverts.org/

Download the tourism.csv file at: https://github.com/elephaint/hierts/tree/main/examples/data

The code for this example can be found at:

* Jupyter Notebook: https://github.com/elephaint/hierts/tree/main/docs/notebooks
* Python script: https://github.com/elephaint/hierts/tree/main/examples

### Import packages & read data

In [1]:
#%% Read packages
import pandas as pd
import numpy as np
from hierts.reconciliation import calc_summing_matrix, apply_reconciliation_methods, \
                                  aggregate_bottom_up_forecasts, calc_level_method_rmse

In [2]:
#%% Read data
df = pd.read_csv('data/tourism.csv', index_col=0)
# We need to convert the Quarter column to a PeriodIndex in order to guarantee correct order
df['Quarter'] = pd.PeriodIndex(df['Quarter'].str[0:4] + '-' + df['Quarter'].str[5:], freq='q')

In [3]:
# Let's look at the data
df.head(10)

,Quarter,Region,State,Purpose,Trips
1,1998Q1,Adelaide,SA,Business,135.077690
2,1998Q2,Adelaide,SA,Business,109.987316
3,1998Q3,Adelaide,SA,Business,166.034687
4,1998Q4,Adelaide,SA,Business,127.160464
5,1999Q1,Adelaide,SA,Business,137.448533
6,1999Q2,Adelaide,SA,Business,199.912586
7,1999Q3,Adelaide,SA,Business,169.355090
8,1999Q4,Adelaide,SA,Business,134.357937
9,2000Q1,Adelaide,SA,Business,154.034398
10,2000Q2,Adelaide,SA,Business,168.776364


As you can see, we have a column indicating the time index ('Quarter'), three columns that provide information on the hierarchy/aggregation of the data ('State', 'Region', 'Purpose'), and a target column ('Trips').

### Set aggregations and calculate summing matrix
For this dataset, the aggregations are in the columns: ['State', 'Region', 'Purpose']. These columns will be in our aggregations.

In [4]:
aggregation_cols = ['State', 'Region', 'Purpose']

Next, we define the aggregations that we are interested in. In this case, we are interested in the following aggregations:

In [5]:
aggregations = [['State'],
                ['State', 'Region'],
                ['State', 'Purpose'],
                ['Purpose']]

Don't include the top (total) level and bottom-level: these will be added automatically later on. Now, we can create a summing matrix, which shows how each of the bottom-level series maps to our chosen aggregations:

In [6]:
df_S = calc_summing_matrix(df, aggregation_cols, aggregations)

Let's have a look at df_S:

In [7]:
df_S

ACT-Canberra-Business  \
Aggregation  Value                                                       
Total        Total                                                 1.0   
State        ACT                                                   1.0   
             NSW                                                   0.0   
             NT                                                    0.0   
             QLD                                                   0.0   
...                                                                ...   
Bottom level WA-Australia's South West-Visiting                    0.0   
             WA-Experience Perth-Business                          0.0   
             WA-Experience Perth-Holiday                           0.0   
             WA-Experience Perth-Other                             0.0   
             WA-Experience Perth-Visiting                          0.0   

                                                 ACT-Canberra-Holiday  \
Aggregation  Value                                                      
Total        Total                                                1.0   
State        ACT                                                  1.0   
             NSW                                                  0.0   
             NT                                                   0.0   
             QLD                                                  0.0   
...                                                               ...   
Bottom level WA-Australia's South West-Visiting                   0.0   
             WA-Experience Perth-Business                         0.0   
             WA-Experience Perth-Holiday                          0.0   
             WA-Experience Perth-Other                            0.0   
             WA-Experience Perth-Visiting                         0.0   

                                                 ACT-Canberra-Other  \
Aggregation  Value                                                    
Total        Total                                              1.0   
State        ACT                                                1.0   
             NSW                                                0.0   
             NT                                                 0.0   
             QLD                                                0.0   
...                                                             ...   
Bottom level WA-Australia's South West-Visiting                 0.0   
             WA-Experience Perth-Business                       0.0   
             WA-Experience Perth-Holiday                        0.0   
             WA-Experience Perth-Other                          0.0   
             WA-Experience Perth-Visiting                       0.0   

                                                 ACT-Canberra-Visiting  \
Aggregation  Value                                                       
Total        Total                                                 1.0   
State        ACT                                                   1.0   
             NSW                                                   0.0   
             NT                                                    0.0   
             QLD                                                   0.0   
...                                                                ...   
Bottom level WA-Australia's South West-Visiting                    0.0   
             WA-Experience Perth-Business                          0.0   
             WA-Experience Perth-Holiday                           0.0   
             WA-Experience Perth-Other                             0.0   
             WA-Experience Perth-Visiting                          0.0   

                                                 NSW-Blue Mountains-Business  \
Aggregation  Value                                                             
Total        Total                                                       1.0   
State    

As you can see, df_S is a mapping between the bottom-level series (columns of df_S) and the aggregations specified. Some observations:

* 'Total' contains the total across all bottom-level series. Hence, this is a row consisting of ones - the total is the sum of all the bottom-level series.
* 'Bottom level' contains the bottom level series. This is an identity matrix, as each bottom-level series in the rows only maps to a single bottom level series in the columns
* Everything in between 'Bottom level' and 'Total' denotes how we can construct the aggregations. For example, the 'State-ACT' aggregation is formed by summing the first four bottom-level series ACT-Canberra-Business, ACT-Canberra-Holiday, ACT-Canberra-Other and ACT-Canberra-Visiting.

### Create a forecasting model for each time series
Now we can create a forecasting model for each time series in the aggregation matrix df_S

In [8]:
# Set target, time_index and split of train and test.
target = 'Trips'
time_index = 'Quarter'
end_train = '2015Q4'
start_test = '2016Q1'

In [9]:
# Add bottom_timeseries identifier and create actuals dataframe for all aggregations
df['bottom_timeseries'] = df[aggregation_cols].agg('-'.join, axis=1)
actuals_bottom_timeseries = df.set_index(['bottom_timeseries', time_index])[target]\
                              .unstack(1)\
                              .loc[df_S.columns]
actuals = df_S @ actuals_bottom_timeseries

Let's look at the actuals:

In [10]:
actuals

Quarter                                                1998Q1        1998Q2  \
Aggregation  Value                                                            
Total        Total                               23182.197269  20323.380067   
State        ACT                                   551.001921    416.025623   
             NSW                                  8039.794795   7166.013805   
             NT                                    181.448823    313.936152   
             QLD                                  4041.370159   3967.904653   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    116.970746    106.044650   
             WA-Experience Perth-Business          187.985188    118.678907   
             WA-Experience Perth-Holiday           294.678011    277.422409   
             WA-Experience Perth-Other              42.414022     35.587768   
             WA-Experience Perth-Visiting          226.134737    237.021622   

Quarter                                                1998Q3        1998Q4  \
Aggregation  Value                                                            
Total        Total                               19826.640511  20830.129891   
State        ACT                                   436.029011    449.798445   
             NSW                                  6747.935790   7282.082371   
             NT                                    528.436859    247.702817   
             QLD                                  4593.893991   4202.829141   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    102.641667    155.877388   
             WA-Experience Perth-Business          129.135987    188.048404   
             WA-Experience Perth-Holiday           250.539302    291.984508   
             WA-Experience Perth-Other              48.137064     61.052428   
             WA-Experience Perth-Visiting          235.038378    291.621175   

Quarter                                                1999Q1        1999Q2  \
Aggregation  Value                                                            
Total        Total                               22087.353380  21458.373285   
State        ACT                                   378.572817    558.178142   
             NSW                                  7584.776839   7054.038705   
             NT                                    184.889592    366.092786   
             QLD                                  4332.490850   4824.480452   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    134.385712    161.531084   
             WA-Experience Perth-Business          164.311015    122.140258   
             WA-Experience Perth-Holiday           338.295122    292.867993   
             WA-Experience Perth-Other              53.626864     71.848886   
             WA-Experience Perth-Visiting          274.072739    267.297383   

Quarter                                                1999Q3        1999Q4  \
Aggregation  Value                                                            
Total        Total                               19914.192508  20027.925640   
State        ACT                                   448.901196    594.825442   
             NSW                                  6723.677146   7135.766945   
             NT                                    501.431284    248.430300   
             QLD                                  5018.034504   4349.835877   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    107.090992    118.579005   
             WA-Experience Perth-Business          148.933004    129.196928   
             WA-Experience Perth-Holiday           274.020511    207.565280   
             WA-Experience Perth-Other         

As can be seen, by taking the dot product of our summing matrix df_S with our bottom-level actuals, we have obtained a matrix where the aggregations are the rows and the timesteps the columns.

We can now proceed to create forecasts.

In [11]:
# Create forecasts: a simple ETS model per timeseries
from sktime.forecasting.ets import AutoETS
forecasts = pd.DataFrame(index=actuals.index, columns = actuals.columns, dtype=np.float32)
for index, series in actuals.iterrows():
    # Fit model and predict (we need to clip because otherwise there's a convergence error)
    model = AutoETS(auto=True, n_jobs=1, random_state=0)
    model.fit(np.clip(series.loc[:end_train], 1e-3, 1e16))
    forecast = model.predict(series.index)
    # Store to forecasts/actuals array
    forecasts.loc[index] = forecast.values

c:\Users\ospra\miniconda3\envs\phd\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Let's look at the forecasts:

In [12]:
forecasts

Quarter                                                1998Q1        1998Q2  \
Aggregation  Value                                                            
Total        Total                               21574.027344  22083.279297   
State        ACT                                   494.419067    500.439117   
             NSW                                  7604.467773   7721.035645   
             NT                                    345.485168    345.468750   
             QLD                                  4156.833496   4131.653809   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    117.684669    117.533379   
             WA-Experience Perth-Business          154.717407    160.191299   
             WA-Experience Perth-Holiday           283.353638    285.672760   
             WA-Experience Perth-Other              46.703793     45.286228   
             WA-Experience Perth-Visiting          231.525131    229.379745   

Quarter                                                1998Q3        1998Q4  \
Aggregation  Value                                                            
Total        Total                               21525.980469  20987.857422   
State        ACT                                   501.933624    502.476624   
             NSW                                  7572.416992   7351.645020   
             NT                                    345.465607    345.483917   
             QLD                                  4095.944336   4204.534668   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    115.098701    112.458824   
             WA-Experience Perth-Business          153.360840    149.374878   
             WA-Experience Perth-Holiday           283.983185    277.134186   
             WA-Experience Perth-Other              42.081352     44.082474   
             WA-Experience Perth-Visiting          232.421234    233.462860   

Quarter                                                1999Q1        1999Q2  \
Aggregation  Value                                                            
Total        Total                               20937.910156  21301.900391   
State        ACT                                   502.227631    498.071075   
             NSW                                  7333.018066   7400.431641   
             NT                                    345.474121    345.458069   
             QLD                                  4204.162598   4232.147949   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    121.660019    124.356834   
             WA-Experience Perth-Business          155.738220    157.148788   
             WA-Experience Perth-Holiday           280.175385    292.077759   
             WA-Experience Perth-Other              49.690231     50.991100   
             WA-Experience Perth-Visiting          256.609985    263.560211   

Quarter                                                1999Q3        1999Q4  \
Aggregation  Value                                                            
Total        Total                               21351.449219  20896.318359   
State        ACT                                   498.810791    496.339600   
             NSW                                  7307.677734   7151.299316   
             NT                                    345.460144    345.475739   
             QLD                                  4361.320312   4504.533203   
...                                                       ...           ...   
Bottom level WA-Australia's South West-Visiting    132.234741    126.906319   
             WA-Experience Perth-Business          151.388489    150.984451   
             WA-Experience Perth-Holiday           292.239594    288.508484   
             WA-Experience Perth-Other         

Observations:

* The AutoETS in this case isn't very powerful: it has mostly created constant forecasts for all timesteps in our test set (see after 2015Q4).
* We create forecasts for both the train and test set. This is because we need the residuals (forecast errors) on our training set.

### Reconciliation
We can now reconcile the forecasts. First, we need to compute the residuals (forecast errors). The residuals are necessary because some reconciliation methods use these. In case you don't have these or it is expensive to obtain them, you are limited to using 'ols' or 'wls_struct' as reconciliation method.

In [13]:
# Calculate residuals. We only need the in-sample (i.e. 'on the training set') residuals.
residuals = (forecasts - actuals)
residuals_train = residuals.loc[:, :end_train]

In [14]:
# Create forecast test set and apply a set of reconciliation methods.
forecasts_test = forecasts.loc[:, start_test:]
forecasts_methods = apply_reconciliation_methods(forecasts_test, df_S, 
                                                residuals_train, 
                                                methods=['ols', 'wls_struct', 'wls_var', 'mint_shrink'])

Method ols, reconciliation time: 0.0023s
Method wls_struct, reconciliation time: 0.0010s
Method wls_var, reconciliation time: 0.0008s
Method mint_shrink, reconciliation time: 0.1298s


We will compare our reconciled forecasts to two cases: (i) the unreconciled forecasts ('base'), and (ii) the bottom-up forecasts ('bottom-up'). The latter is obtained by merely summing our bottom-level forecasts for all the aggregations (i.e. we effectively dispense with the separate forecasts for all the higher-level aggregations).

In [15]:
# Create the bottom-up forecasts and add them to the forecasts_methods dataframe
forecasts_bu_bottom_level = forecasts.loc['Bottom level']
forecasts_bu = aggregate_bottom_up_forecasts(forecasts_bu_bottom_level, df_S)
forecasts_bu_test = forecasts_bu.loc[:, start_test:]
forecasts_method = pd.concat({'bottom-up': forecasts_bu_test}, names=['Method'])
forecasts_methods = pd.concat((forecasts_method, forecasts_methods), axis=0)

Finally, we can compute the root mean-squared error on all of our methods.

In [18]:
# Calculate error for all levels and methods. We set bottom-up as the base method to compare against
# in the relative rmse
rmse, rel_rmse = calc_level_method_rmse(forecasts_methods, actuals, base='base')

Let's look at the errors:

In [19]:
rmse

Method,bottom-up,base,ols,wls_struct,wls_var,mint_shrink
Aggregation,,,,,,
Total,3306.986430,2384.445914,2437.109309,2699.844643,2794.340494,1633.549770
Purpose,929.189979,749.342014,723.644402,797.040328,818.227326,569.388713
State,654.361254,567.977400,560.540256,595.149739,604.404402,447.350145
State-Purpose,190.521026,174.316176,169.223200,175.491348,176.560629,137.542250
State-Region,93.707038,94.867123,87.760903,89.019715,87.381595,76.314683
Bottom level,36.514784,36.514784,35.654509,35.745782,35.367992,33.347415
All series,217.198623,172.006112,171.058750,185.599777,190.150964,127.900052


We can see the following:

* The base forecast, where we create a separate forecast for each time series in each aggregation, has an overall RMSE of 172.
* The base forecast has been reconciled using 'ols', 'wls_struct', 'wls_var' and 'mint_shrink'. Of these 4 methods, 'mint_shrink' achieves superior performance; across all levels the forecast is better than the base forecast.
* The bottom-up forecast performs rather poorly. Note that (by definition) base == bottom-up for the Bottom level aggregation.

Finally, let's look at the relative rmse. This provides an easier insight into which method performs best. We compare against the base forecast.

In [20]:
rel_rmse

Method,bottom-up,base,ols,wls_struct,wls_var,mint_shrink
Aggregation,,,,,,
Total,1.386899,1.0,1.022086,1.132273,1.171903,0.685086
Purpose,1.240008,1.0,0.965706,1.063654,1.091928,0.759852
State,1.152090,1.0,0.986906,1.047841,1.064135,0.787620
State-Purpose,1.092962,1.0,0.970783,1.006742,1.012876,0.789039
State-Region,0.987771,1.0,0.925093,0.938362,0.921095,0.804438
Bottom level,1.000000,1.0,0.976440,0.978940,0.968594,0.913258
All series,1.262738,1.0,0.994492,1.079030,1.105490,0.743579


Clearly, 'mint_shrink' provides the best performance, across all series it performs about 25% better than the base forecast.